# Подключение gdrive(rclone)

In [ ]:
!wget https://downloads.rclone.org/v1.53.1/rclone-v1.53.1-linux-amd64.deb 
!apt install ./rclone-v1.53.1-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/drive
!nohup rclone --vfs-cache-mode writes mount  drive:  /content/drive &

# Подключение gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Инициализация

In [1]:
!pip install -q transformers sentencepiece sentence-transformers catboost simpletransformers catboost

In [2]:
import pandas as pd
%matplotlib inline
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from torch import cuda
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
from simpletransformers.classification import ClassificationModel
from sentence_transformers import SentenceTransformer
import os
import re
import pickle
import time
import pylab as pl
import matplotlib.pyplot as plt
from IPython import display
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import nltk
from sklearn.model_selection import KFold
from nltk.corpus import stopwords
from catboost import CatBoostRegressor
import gensim
from gensim.utils import simple_preprocess
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
def create_folds(data, target, num_splits=3):
    if num_splits > 1:
        data.loc[:,'kfold'] = -1
        X = data.drop([target], axis=1)
        y = data[[target]]
        mskf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
        for fold, (trn_, val_) in enumerate(mskf.split(data)):
            data.loc[val_,'kfold'] = fold
    else:
        data.loc[:,'kfold'] = 0
        

    return data

# Изучение данных(train_issues)

In [4]:
train_comments  = pd.read_csv("/content/drive/MyDrive/ALTGU/train_comments.csv").dropna()
train_issues = pd.read_csv("/content/drive/MyDrive/ALTGU/train_issues.csv").dropna()

In [5]:
fig = px.box(train_issues, y="overall_worklogs")
fig.show()

In [6]:
train_issues = train_issues[train_issues.overall_worklogs < 90000]

In [7]:
fig = px.histogram(train_issues, y="overall_worklogs", x="created", histfunc="avg")
fig.show()

In [8]:
fig = px.histogram(train_issues, y="overall_worklogs", x="project_id", histfunc="avg")
fig.show()

In [9]:
fig = px.histogram(train_issues, y="overall_worklogs", x="assignee_id", histfunc="avg", nbins=int(train_issues.assignee_id.max()))
fig.show()

#  Изучение данных(train_comments)

In [ ]:
data = {}
for i in train_comments.issue_id.to_list():
  if i not in data:
    data[i] = 0
  data[i]+=1
print(sorted(list(data.items()), key=lambda x:-x[1]))

In [11]:
def tok(text):
  text = re.sub("\(.*\)", "", text)
  text = re.sub("\{.*\}", "", text)
  text = re.sub("\[.*\]", "", text)
  return re.sub("[^A-Za-z0-9 ]", "", text)

In [12]:
train_comments.text = train_comments.text.apply(tok)

In [ ]:
data = {}
for i, row in train_comments.iterrows():
  if row[2] not in data:
    data[row[2]] = 0
  data[row[2]]  += len(row[1].split())
print(sorted(list(data.items()), key=lambda x:-x[1]))

In [ ]:
data = {}
for i, row in train_comments.iterrows():
  if row[2] not in data:
    data[row[2]] = []
  if row[3] not in data[row[2]]:
    data[row[2]].append(row[3])
print(sorted(list(data.items()), key=lambda x:-len(x[1])))

# Изучение данных(employees)

In [15]:
employees = pd.read_csv("/content/drive/MyDrive/ALTGU/employees.csv")

In [ ]:
employees

In [17]:
employees.active.sum() / employees.shape[0]

0.49271137026239065

In [18]:
employees.position.isnull().sum() / employees.position.shape[0]

0.43731778425655976

In [19]:
employees.hiring_type.isnull().sum() / employees.hiring_type.shape[0]

0.24198250728862974

In [20]:
employees.payment_type.isnull().sum() / employees.payment_type.shape[0]

0.3556851311953353

In [21]:
employees.salary_calculation_type.isnull().sum() / employees.salary_calculation_type.shape[0]

0.9037900874635568

In [22]:
employees.english_level.isnull().sum() / employees.english_level.shape[0]

0.9533527696793003

In [23]:
employees.passport.sum() / employees.passport.shape[0]

0.46355685131195334

In [24]:
employees.is_labor_contract_signed.sum() / employees.is_labor_contract_signed.shape[0]

0.22448979591836735

In [25]:
employees.is_added_to_internal_chats.sum() / employees.is_added_to_internal_chats.shape[0]

0.2536443148688047

In [26]:
employees.is_added_one_to_one.sum() / employees.is_added_one_to_one.shape[0]

0.17784256559766765

# Преобрзование(train)

In [27]:
train_comments  = pd.read_csv("/content/drive/MyDrive/ALTGU/train_comments.csv").dropna()
train_issues = pd.read_csv("/content/drive/MyDrive/ALTGU/train_issues.csv").dropna()

In [28]:
train_issues.key  = train_issues.key.apply(lambda x: x.split("-")[0])
train_issues = train_issues.drop(columns=["created", "project_id"])

In [ ]:
data = {}
for i in train_comments.issue_id.to_list():
  if i not in data:
    data[i] = 0
  data[i]+=1
print(sorted(list(data.items()), key=lambda x:-x[1]))

In [30]:
final ={"id":[],"count_com":[]}
for i, j in data.items():
  final["id"].append(i)
  final["count_com"].append(j)
df = pd.DataFrame(final)

In [31]:
df3 = pd.merge(train_issues,df,on='id',how='left').fillna(0).drop(columns=["id"])

In [32]:
df3.to_csv("train.csv")

# Преобрзование(test)

In [33]:
test_comments  = pd.read_csv("/content/drive/MyDrive/ALTGU/test_comments.csv").dropna()
test_issues = pd.read_csv("/content/drive/MyDrive/ALTGU/test_issues.csv").dropna()
test_issues.key  = test_issues.key.apply(lambda x: x.split("-")[0])
test_issues = test_issues.drop(columns=["created", "project_id"])

In [ ]:
data = {}
for i in test_comments.issue_id.to_list():
  if i not in data:
    data[i] = 0
  data[i]+=1
print(sorted(list(data.items()), key=lambda x:-x[1]))

In [35]:
final ={"id":[],"count_com":[]}
for i, j in data.items():
  final["id"].append(i)
  final["count_com"].append(j)
df = pd.DataFrame(final)

In [36]:
df3_test = pd.merge(test_issues,df,on='id',how='left').fillna(0).drop(columns=["id"])

# Эмбидинги для train

In [43]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].detach().cpu() #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def make_features_transformers(df, model_name, df_model, col, max_len, check=False):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  if check:
    model =torch.load(model_name)
  else:
    model = AutoModel.from_pretrained(model_name).cuda()
  text_features = []
  for sentence in tqdm(df[col]):
    encoded_input = tokenizer([sentence], padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')
    with torch.no_grad():
        model_output = model(input_ids=encoded_input['input_ids'].cuda())
    sentence_embeddings = list(mean_pooling(model_output, encoded_input['attention_mask']).numpy())
      #print(model_output)
    #print(sentence_embeddings)
    text_features.extend(sentence_embeddings)
  text_features_df = pd.DataFrame(text_features, columns = [f'{df_model}_{col}_feature_{i}' for i in range(len(text_features[0]))])
  return text_features_df

In [ ]:
models = [
     ('facebook/bart-large-mnli', 128),
     ("ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli", 128),  
     ("joeddav/xlm-roberta-large-xnli", 128),
     ('microsoft/mdeberta-v3-base',  128),
     ('MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',  128)
     ]
for m in models:
  print(m)
  df3 = df3.join(make_features_transformers(df3, m[0], m[0].split('/')[1], 'summary', m[1]))
  df3.to_csv('altgu_transformers_text_features_base.csv', index=False)

In [50]:
def make_labse(df, col):
  df_model = 'labse'
  model = SentenceTransformer('sentence-transformers/LaBSE')
  text_features = []
  for sentence in tqdm(df[col]):
    sentence_embeddings = list(model.encode([sentence]))
    text_features.extend(sentence_embeddings)
  text_features_df = pd.DataFrame(text_features, columns = [f'{df_model}_{col}_feature_{i}' for i in range(len(text_features[0]))])
  return text_features_df

In [ ]:
df3 = df3.join(make_labse(df3, 'summary'))
df3.to_csv('altgu_transformers_text_features_base.csv', index=False)

# Эмбидинги для test

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].detach().cpu() #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def make_features_transformers(df, model_name, df_model, col, max_len):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModel.from_pretrained(model_name).cuda()
  text_features = []
  for sentence in tqdm(df[col]):
    encoded_input = tokenizer([sentence], padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')
    with torch.no_grad():
      model_output = model(input_ids=encoded_input['input_ids'].cuda())
      #print(model_output)
    sentence_embeddings = list(mean_pooling(model_output, encoded_input['attention_mask']).numpy())
    #print(sentence_embeddings)
    text_features.extend(sentence_embeddings)
  text_features_df = pd.DataFrame(text_features, columns = [f'{df_model}_{col}_feature_{i}' for i in range(len(text_features[0]))])
  return text_features_df

In [ ]:
models = [
     ('facebook/bart-large-mnli', 128),
     ("ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli", 128),  
     ("joeddav/xlm-roberta-large-xnli", 128),
     ('microsoft/mdeberta-v3-base',  128),
     ('MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',  128)
     ]
for m in models:
  print(m)
  df3_test = df3_test.join(make_features_transformers(df3_test, m[0], m[0].split('/')[1], 'summary', m[1]))
  df3_test.to_csv('altgu_transformers_text_test_base.csv', index=False)

In [ ]:
def make_labse(df, col):
  df_model = 'labse'
  model = SentenceTransformer('sentence-transformers/LaBSE')
  text_features = []
  for sentence in tqdm(df[col]):
    sentence_embeddings = list(model.encode([sentence]))
    text_features.extend(sentence_embeddings)
  text_features_df = pd.DataFrame(text_features, columns = [f'{df_model}_{col}_feature_{i}' for i in range(len(text_features[0]))])
  return text_features_df

In [ ]:
df3_test = df3_test.join(make_labse(df3_test, 'summary'))
df3_test.to_csv('altgu_transformers_text_test_base.csv', index=False)

# Отбор признаков

In [37]:
df3 = pd.read_csv("/content/drive/MyDrive/rbk_transformers_text_features_base.csv")

In [38]:
y_train = df3["overall_worklogs"]
X_train = df3.drop(columns=["overall_worklogs","summary"])

In [ ]:
reg = CatBoostRegressor(iterations=500).fit(X_train, y_train, cat_features=['key', 'assignee_id', 'creator_id'])

In [ ]:
f_i = pd.DataFrame({'feature_importance': reg.feature_importances_, 
              'feature_names': X_train.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
f_i.to_csv('ALTGU_features.csv', index=False)

In [ ]:
f_i[f_i.feature_importance > 0.001].head(20)

# Обучение catboost

In [40]:
need_features = pd.read_csv('/content/ALTGU_features.csv').iloc[:1000]
train = pd.read_csv("/content/drive/MyDrive/rbk_transformers_text_features_base.csv")[need_features.feature_names]
test = pd.read_csv("/content/drive/MyDrive/rbk_transformers_text_test_base.csv")[need_features.feature_names]

In [41]:
train_issues = pd.read_csv("/content/drive/MyDrive/ALTGU/train_issues.csv").dropna()

In [42]:
train['overall_worklogs'] = train_issues["overall_worklogs"]

In [43]:
df = train
df = create_folds(df, "overall_worklogs", num_splits=5)

In [44]:
target_features = ["overall_worklogs"]
X = df.drop(columns=target_features)
y = df[['overall_worklogs', 'kfold']]

In [ ]:
regs = []
scores = []
N = 5
val_ds = pd.DataFrame({'true': y['overall_worklogs']})
val_ds['pred'] = -1
for kfold in range(5):
    X_train, X_test = X[X.kfold!=kfold].drop('kfold', axis=1), X[X.kfold==kfold].drop('kfold', axis=1)
    y_train, y_test = y[y.kfold!=kfold].drop('kfold', axis=1), y[y.kfold==kfold].drop('kfold', axis=1)
    print(X_train.shape)
    print(f'--------------------------------{kfold}-fold-------------------------------')
    reg = CatBoostRegressor(iterations = 2000,
                          loss_function='RMSE',
                          l2_leaf_reg=2,
                          depth=10,                          
                          task_type='GPU', eval_metric='R2',
                          #min_data_in_leaf = 10,
                          #max_bin = 220,
                          #verbose=100,
                          #leaf_estimation_iterations=30,
                          od_wait=500,
                          grow_policy='Lossguide',
                          learning_rate=0.05,
                          #bootstrap_type='Poisson',
                          verbose=10
                          )
    
    reg.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model = True, plot = False, cat_features=['key', 'assignee_id', 'creator_id'])
    scores.append(r2_score(y_test, reg.predict(X_test)))
    val_ds.loc[y_test.index, 'pred'] = reg.predict(X_test)
    regs.append(reg)
        
print(scores)
assert len(regs) == N

print('mean R2', r2_score(val_ds['true'], val_ds['pred']))

# Создание сабмита

In [ ]:
submit = pd.DataFrame()
submit["id"] = pd.read_csv("/content/drive/MyDrive/ALTGU/test_issues.csv").id
y_pred = 0
for reg in regs:
    y_pred += reg.predict(test)#[:,1]
y_pred /= N
submit['overall_worklogs'] = y_pred
print('mean R2', r2_score(val_ds['true'], val_ds['pred']))

In [47]:
submit.to_csv("sumbit.csv", index=False)